In [26]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import googlemaps
import gmplot

from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('API_KEY')

In [20]:
gmaps = googlemaps.Client(API_KEY)

In [21]:
end1 = 'Dr. Paulo Tinoco Cabral 935, Ribeirão Preto, SP'
end2 = 'Rua João Tapetti 256, Ribeirão Preto, SP'
end3 = 'Avenida Presidente Vargas 1000, Ribeirão Preto, SP'
end4 = 'Rua José Gatti 100, Ribeirão Preto, SP'
end5 = 'Avenida Maurilio Biagi 1011, Ribeirão Preto, SP'

end_list = [end1, end2, end3, end4, end5]

In [22]:
coordinates = []

lats = []
lgns = []

for end in end_list:
    geocode_result = gmaps.geocode(end)
    coordinates.append(geocode_result[0]['geometry']['location'])
    
    lats.append(geocode_result[0]['geometry']['location']['lat'])
    lgns.append(geocode_result[0]['geometry']['location']['lng'])

In [23]:
apikey = API_KEY

gmap = gmplot.GoogleMapPlotter(
    np.mean(lats),
    np.mean(lgns),
    13,
    apikey=apikey
)

gmap.scatter(lats, lgns, color='red')
gmap.draw('locations.html')

In [6]:
df_coords = pd.DataFrame(coordinates)
df_coords.columns = ['x', 'y']
df_coords['cod_fazenda'] = df_coords.index

In [7]:
df_coords

,x,y,cod_fazenda
0,-21.201437,-47.807650,0
1,-21.131067,-47.829862,1
2,-21.197431,-47.808127,2
3,-21.169402,-47.811085,3
4,-21.202385,-47.794576,4


In [8]:
import numpy as np
import time

from src.io_helper import read_tsp, normalize, generate_tsp_from_excel
from src.neuron import generate_network, get_neighborhood
from src.distance import select_closest, route_distance, get_route
from src.plot import plot_network, plot_route
from src.gif import get_frames, create_gif, remove_images

In [9]:
def som(problem, iterations, filename, learning_rate=0.8):
    """Solve the TSP using a Self-Organizing Map."""
    
    # Obtain the normalized set of cities (w/ coord in [0,1])
    cities = problem.copy()

    cities[['x', 'y']] = normalize(cities[['x', 'y']])

    # The population size is 8 times the number of cities
    n = cities.shape[0] * 8

    # Generate an adequate network of neurons:
    network = generate_network(n)
    print('Network of {} neurons created. Starting the iterations:'.format(n))

    for i in range(iterations):
        if not i % 100:
            print('\t> Iteration {}/{}'.format(i, iterations), end="\r")
        # Choose a random city
        city = cities.sample(1)[['x', 'y']].values
        winner_idx = select_closest(network, city)
        # Generate a filter that applies changes to the winner's gaussian
        gaussian = get_neighborhood(winner_idx, n//10, network.shape[0])
        # Update the network's weights (closer to the city)
        network += gaussian[:,np.newaxis] * learning_rate * (city - network)
        # Decay the variables
        learning_rate = learning_rate * 0.99997
        n = n * 0.9997

        # Check for plotting interval
        if not i % 1000:
            plot_network(cities, network, name='diagrams/{:05d}.jpg'.format(i))

        # Check if any parameter has completely decayed.
        if n < 1:
            print('Radius has completely decayed, finishing execution',
            'at {} iterations'.format(i))
            break
        if learning_rate < 0.001:
            print('Learning rate has completely decayed, finishing execution',
            'at {} iterations'.format(i))
            break
    else:
        print('Completed {} iterations.'.format(iterations))

    plot_network(cities, network, name='diagrams/final.jpg')

    route = get_route(cities, network)
    plot_route(cities, route, f'diagrams/route_{filename}.jpg')
    
    return route

In [10]:
route = som(df_coords, 100000, 'teste')

Network of 40 neurons created. Starting the iterations:
Radius has completely decayed, finishing execution at 12294 iterations


In [11]:
route

Int64Index([2, 0, 4, 3, 1], dtype='int64')

In [12]:
start = 0
lst_route = list(route)

idx = lst_route.index(start)

last = lst_route[idx:]
first = lst_route[:idx]

final = last + first

In [13]:
final

[0, 4, 3, 1, 2]

In [14]:
for i, (_, row) in enumerate(df_coords.iloc[final].iterrows()):
    print(i, row)

0 x             -21.201437
y             -47.807650
cod_fazenda     0.000000
Name: 0, dtype: float64
1 x             -21.202385
y             -47.794576
cod_fazenda     4.000000
Name: 4, dtype: float64
2 x             -21.169402
y             -47.811085
cod_fazenda     3.000000
Name: 3, dtype: float64
3 x             -21.131067
y             -47.829862
cod_fazenda     1.000000
Name: 1, dtype: float64
4 x             -21.197431
y             -47.808127
cod_fazenda     2.000000
Name: 2, dtype: float64


In [6]:


apikey = API_KEY
gmap = gmplot.GoogleMapPlotter(
    -21.131067,
    -47.811085,
    13,
    apikey=apikey
)

gmap.directions(
    (-21.131067, -47.829862),
    (-21.169402, -47.811085),
    waypoints=[
        (-21.197431, -47.808127),
        (-21.201437, -47.807650),
        (-21.202385, -47.794576),
    ]
)

gmap.draw('map.html')